In [170]:
import cv2
import numpy as np
import matplotlib

print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)

OpenCV version: 4.10.0
NumPy version: 1.26.4
Matplotlib version: 3.9.0


In [171]:
# import cv2
# import numpy as np

# def detect_and_save_kicker_upper(image_path, save_path):
#     # Load the image
#     image = cv2.imread(image_path)
#     if image is None:
#         print("Error: Image not found.")
#         return None

#     # Step 1: Image Preprocessing
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

#     # Step 2: Edge Detection
#     edges = cv2.Canny(blurred_image, 50, 150)

#     # Step 3: Define ROI (upper half of the image)
#     height, width = edges.shape
#     roi = edges[0:int(height / 2), 0:width]

#     # Step 4: Save the resulting ROI image
#     cv2.imwrite(save_path, roi)

#     return roi

# # Example usage:
# # roi_image = detect_and_save_kicker_upper("path/to/your/image.jpg", "path/to/save/roi_image.jpg")


# # Example usage
# upper_roi = detect_and_save_kicker_upper('False_images_class1/cam2024_09_26_17_23_59.jpg', 'Reference_Imgs/invalid noisy roi.jpg')

In [172]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to detect kicker using white pixel coverage
def detect_kicker_by_white_pixels(image_path, threshold_percentage=5, threshold_value=127):
    # Load the input image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("Error: Image not found.")
        return False

    # Apply binary threshold to convert the image to black and white
    _, img_bw = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)

    # Focus only on the upper half of the image (ROI where kicker might be)
    height, width = img.shape
    upper_region = img_bw[0:int(height / 2), :]  # Upper half of the image

    # Calculate the total number of pixels in the upper region
    total_pixels = upper_region.size

    # Count the number of white pixels (255) in the upper region
    white_pixels = np.sum(upper_region == 255)

    # Calculate the percentage of white pixels
    white_pixel_percentage = (white_pixels / total_pixels) * 100

    # Visualize the upper region and the binary image
    # plt.subplot(121), plt.imshow(img_bw, cmap='gray')
    # plt.title('Binary Image (BW)'), plt.xticks([]), plt.yticks([])
    # plt.subplot(122), plt.imshow(upper_region, cmap='gray')
    # plt.title('Upper Region'), plt.xticks([]), plt.yticks([])
    # plt.show()

    # Print the percentage of white pixels
    print(f'White Pixel Percentage in Upper Region: {white_pixel_percentage:.2f}%')

    # Check if the percentage exceeds the threshold for detecting the kicker
    if white_pixel_percentage > threshold_percentage:
        print("Kicker Detected by White Pixels")
        return True
    else:
        print("No Kicker Detected by White Pixels")
        return False


# Step 1: Kicker Detection
def detect_kicker(image_path, min_line_count=2, min_intersect_count=1, white_pixel_threshold_percentage=5):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return None, None

    # Step 1: Image Preprocessing
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Step 2: Edge Detection
    edges = cv2.Canny(blurred_image, 50, 150)

    # Step 3: Define ROI (upper half of the image)
    height, width = edges.shape
    roi = edges[0:int(height/2), 0:width]

    # Step 4: Pattern Recognition (Hough Transform for lines)
    lines = cv2.HoughLines(roi, 1, np.pi/180, 100)

    # Step 5: Analyze Detected Lines
    kicker_detected = False
    line_image = np.zeros_like(image)
    line_count = 0
    intersect_count = 0
    line_data = []

    if lines is not None:
        for rho, theta in lines[:, 0]:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Store line data for intersection check
            line_data.append(((x1, y1), (x2, y2)))
            line_count += 1

    # Step 6: Check if sufficient lines are detected
    if line_count >= min_line_count:
        # Step 7: Check for line intersections to detect kicker-like patterns
        for i in range(len(line_data)):
            for j in range(i + 1, len(line_data)):
                if check_line_intersection(line_data[i], line_data[j]):
                    intersect_count += 1

    # If enough intersections are found, we assume a kicker is detected
    kicker_detected = (line_count >= min_line_count) and (intersect_count >= min_intersect_count)

    # # Visualization
    # plt.figure(figsize=(12, 10))

    # # Edges
    # plt.subplot(2, 3, 4)
    # plt.imshow(edges, cmap='gray')
    # plt.title("Detected Edges")
    # plt.axis("off")

    # # ROI
    # #cv2.imwrite("ROI.jpg",roi)
    # plt.subplot(2, 3, 5)
    # plt.imshow(roi, cmap='gray')
    # plt.title("ROI (Upper Half)")
    # plt.axis("off")

    # # Detected Lines (if any)
    # plt.subplot(2, 3, 6)
    # plt.imshow(cv2.cvtColor(line_image, cv2.COLOR_BGR2RGB))
    # if kicker_detected:
    #     plt.title(f"(Lines: {line_count}, Intersections: {intersect_count})")
    # else:
    #     plt.title("No Kicker Detected")
    # plt.axis("off")

    # plt.tight_layout()
    # plt.show()

    # Check both line detection and white pixel coverage
    if kicker_detected:
        print("Test 1 successful. Checking white pixel coverage...")
        if not detect_kicker_by_white_pixels(image_path, threshold_percentage=white_pixel_threshold_percentage):
            print("Test 2 failed. No kicker detected (due to insufficient white pixels).")
            return None, None
        else:
            print("Kicker detected (both tests passed).")
            return roi, edges  # Both tests passed
    else:
        print("Test 1 failed. Checking white pixel coverage...")
        if detect_kicker_by_white_pixels(image_path, threshold_percentage=white_pixel_threshold_percentage):
            print("Kicker detected (based on white pixel coverage).")
            return roi, edges  # Kicker detected based on white pixel coverage
        else:
            print("All tests failed. No kicker detected.")
            return None, None


# Function to check if two lines intersect
def check_line_intersection(line1, line2):
    def ccw(A, B, C):
        return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])

    (A, B) = line1
    (C, D) = line2
    return ccw(A, C, D) != ccw(B, C, D) and ccw(A, B, C) != ccw(A, B, D)


### Multiple checks 

In [173]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# # Function to extract the central vertical region from an image
# def extract_vertical_central_region(image, width_ratio=0.3, height_ratio=0.8):
#     height, width = image.shape
#     central_height = int(height * height_ratio)
#     central_width = int(width * width_ratio)
    
#     start_y = (height - central_height) // 2
#     start_x = (width - central_width) // 2
    
#     return image[start_y:start_y + central_height, start_x:start_x + central_width]

# # Function for template matching
# def template_matching(current_central, valid_central, invalid_central):
#     valid_result = cv2.matchTemplate(current_central, valid_central, cv2.TM_CCOEFF_NORMED)
#     invalid_result = cv2.matchTemplate(current_central, invalid_central, cv2.TM_CCOEFF_NORMED)

#     valid_match = np.max(valid_result)
#     invalid_match = np.max(invalid_result)

#     return valid_match, invalid_match

# # Function for feature matching using ORB
# def feature_matching(current_central, valid_central, invalid_central):
#     orb = cv2.ORB_create()
#     keypoints1, descriptors1 = orb.detectAndCompute(current_central, None)
#     keypoints2, descriptors2 = orb.detectAndCompute(valid_central, None)
#     keypoints3, descriptors3 = orb.detectAndCompute(invalid_central, None)

#     # Use BFMatcher for matching
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
#     matches_valid = bf.match(descriptors1, descriptors2)
#     matches_invalid = bf.match(descriptors1, descriptors3)

#     valid_similarity = len(matches_valid)
#     invalid_similarity = len(matches_invalid)

#     return valid_similarity, invalid_similarity

# # Function to check kicker orientation based on template and feature matching
# def check_kicker_orientation(current_roi, valid_roi_path, invalid_roi_path, width_ratio=0.3, height_ratio=0.8):
#     valid_roi = cv2.imread(valid_roi_path, cv2.IMREAD_GRAYSCALE)
#     invalid_roi = cv2.imread(invalid_roi_path, cv2.IMREAD_GRAYSCALE)
    
#     if valid_roi is None or invalid_roi is None:
#         print("Error: One or both reference ROIs not found.")
#         return False

#     # Extract central vertical regions
#     current_central = extract_vertical_central_region(current_roi, width_ratio, height_ratio)
#     valid_central = extract_vertical_central_region(valid_roi, width_ratio, height_ratio)
#     invalid_central = extract_vertical_central_region(invalid_roi, width_ratio, height_ratio)

#     # Test 1: Template Matching
#     valid_match, invalid_match = template_matching(current_central, valid_central, invalid_central)

#     # Test 2: Feature Matching
#     valid_similarity, invalid_similarity = feature_matching(current_central, valid_central, invalid_central)

#     # Visualize results
#     plt.figure(figsize=(18, 6))
    
#     plt.subplot(1, 3, 1)
#     plt.imshow(current_central, cmap='gray')
#     plt.title("Current ROI Vertical Region")
#     plt.axis("off")

#     plt.subplot(1, 3, 2)
#     plt.title(f"Template Match Valid: {valid_match:.4f}, Feature Matches: {valid_similarity}")
#     plt.axis("off")

#     plt.subplot(1, 3, 3)
#     plt.title(f"Template Match Invalid: {invalid_match:.4f}, Feature Matches: {invalid_similarity}")
#     plt.axis("off")

#     plt.tight_layout()
#     plt.show()

#     # Decision Logic
#     if valid_match > invalid_match and valid_similarity > invalid_similarity:
#         print("Kicker is in a valid orientation.")
#         return True
#     else:
#         print("Kicker is in an invalid orientation.")
#         return False

# # Full pipeline function
# def kicker_pipeline(image_path, valid_roi_path, invalid_roi_path, threshold=65, white_pixel_threshold_percentage=5):
#     # Step 1: Detect Kicker
#     current_roi, current_edges = detect_kicker(image_path, white_pixel_threshold_percentage=white_pixel_threshold_percentage)
#     if current_roi is None:
#         return False  # No kicker detected

#     # Step 2: Check Orientation if kicker detected
#     return check_kicker_orientation(current_roi, valid_roi_path, invalid_roi_path)


In [174]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# # Function to extract the central vertical region from an image
# def extract_vertical_central_region(image, width_ratio=0.3, height_ratio=0.8):
#     height, width = image.shape
#     central_height = int(height * height_ratio)
#     central_width = int(width * width_ratio)
    
#     start_y = (height - central_height) // 2
#     start_x = (width - central_width) // 2
    
#     return image[start_y:start_y + central_height, start_x:start_x + central_width]

# # Function for template matching
# def template_matching(current_central, reference_central):
#     result = cv2.matchTemplate(current_central, reference_central, cv2.TM_CCOEFF_NORMED)
#     match_score = np.max(result)
#     return match_score

# # Function for feature matching using ORB
# def feature_matching(current_central, reference_central):
#     orb = cv2.ORB_create()
#     keypoints1, descriptors1 = orb.detectAndCompute(current_central, None)
#     keypoints2, descriptors2 = orb.detectAndCompute(reference_central, None)

#     # Use BFMatcher for matching
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
#     matches = bf.match(descriptors1, descriptors2)
    
#     return len(matches)  # Number of matches

# # Function to check kicker orientation based on multiple reference images
# def check_kicker_orientation(current_roi, valid_roi_paths, width_ratio=0.3, height_ratio=0.8):
#     # Load all reference ROIs
#     valid_rois = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in valid_roi_paths]
    
#     # Check if any ROI is None
#     if any(roi is None for roi in valid_rois):
#         print("Error: One or more reference ROIs not found.")
#         return False

#     # Extract central vertical regions
#     current_central = extract_vertical_central_region(current_roi, width_ratio, height_ratio)
#     central_regions = [extract_vertical_central_region(roi, width_ratio, height_ratio) for roi in valid_rois]

#     # Initialize scores
#     scores = []

#     # Compare current central region with all reference central regions
#     for reference_central in central_regions:
#         template_score = template_matching(current_central, reference_central)
#         feature_score = feature_matching(current_central, reference_central)
#         scores.append((template_score, feature_score))

#     # Visualize results
#     plt.figure(figsize=(12, 6))
    
#     plt.subplot(1, 2, 1)
#     plt.imshow(current_central, cmap='gray')
#     plt.title("Current ROI Vertical Region")
#     plt.axis("off")

#     plt.subplot(1, 2, 2)
#     for i, (template_score, feature_score) in enumerate(scores):
#         plt.text(0, i * 0.1, f"ROI {i+1}: Template Score: {template_score:.4f}, Feature Matches: {feature_score}", fontsize=12)
    
#     plt.axis("off")
#     plt.tight_layout()
#     plt.show()

#     # Decision Logic: Choose the best score
#     best_match_index = np.argmax([score[0] for score in scores])
#     best_template_score = scores[best_match_index][0]
#     best_feature_score = scores[best_match_index][1]

#     # Determine if the best score corresponds to a valid or invalid orientation
#     if best_match_index in [0, 1]:  # Assuming 0 and 1 are valid indices
#         print("Kicker is in a valid orientation.")
#         return True
#     else:
#         print("Kicker is in an invalid orientation.")
#         return False

# # Full pipeline function
# def kicker_pipeline(image_path, valid_roi_paths, threshold=65, white_pixel_threshold_percentage=5):
#     # Step 1: Detect Kicker
#     current_roi, current_edges = detect_kicker(image_path, white_pixel_threshold_percentage=white_pixel_threshold_percentage)
#     if current_roi is None:
#         return False  # No kicker detected

#     # Step 2: Check Orientation if kicker detected
#     return check_kicker_orientation(current_roi, valid_roi_paths)

# # Example usage
# valid_roi_paths = [
#     'Reference_Imgs/valid clean roi.jpg',
#     'Reference_Imgs/valid noisy roi.jpg',
#     'Reference_Imgs/invalid clean roi.jpg',
#     'Reference_Imgs/invalid noisy roi.jpg'
# ]

# result = kicker_pipeline('OneDrive_2024-10-01/Images for Testing/Class 1 - Kicker at front position/cam2024_10_01_10_20_54.jpg', valid_roi_paths)


In [175]:
# import cv2
# import numpy as np

# # Function to extract the central vertical region from an image
# def extract_vertical_central_region(image, width_ratio=0.3, height_ratio=0.8):
#     height, width = image.shape
#     central_height = int(height * height_ratio)
#     central_width = int(width * width_ratio)
    
#     start_y = (height - central_height) // 2
#     start_x = (width - central_width) // 2
    
#     return image[start_y:start_y + central_height, start_x:start_x + central_width]

# # Function for template matching
# def template_matching(current_central, reference_central):
#     result = cv2.matchTemplate(current_central, reference_central, cv2.TM_CCOEFF_NORMED)
#     match_score = np.max(result)
#     return match_score

# # Function for feature matching using ORB
# def feature_matching(current_central, reference_central):
#     orb = cv2.ORB_create()
#     keypoints1, descriptors1 = orb.detectAndCompute(current_central, None)
#     keypoints2, descriptors2 = orb.detectAndCompute(reference_central, None)

#     # Use BFMatcher for matching
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
#     matches = bf.match(descriptors1, descriptors2)
    
#     return len(matches)  # Number of matches

# # Function to check kicker orientation based on multiple reference images
# def check_kicker_orientation(current_roi, valid_roi_paths, width_ratio=0.3, height_ratio=0.8):
#     # Load all reference ROIs
#     valid_rois = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in valid_roi_paths]
    
#     # Check if any ROI is None
#     if any(roi is None for roi in valid_rois):
#         print("Error: One or more reference ROIs not found.")
#         return False

#     # Extract central vertical regions
#     current_central = extract_vertical_central_region(current_roi, width_ratio, height_ratio)
#     central_regions = [extract_vertical_central_region(roi, width_ratio, height_ratio) for roi in valid_rois]

#     # Initialize scores
#     scores = []

#     # Compare current central region with all reference central regions
#     roi_descriptions = [
#         "clean valid ROI",
#         "noisy valid ROI",
#         "clean invalid ROI",
#         "noisy invalid ROI"
#     ]

#     for i, reference_central in enumerate(central_regions):
#         template_score = template_matching(current_central, reference_central)
#         feature_score = feature_matching(current_central, reference_central)
#         scores.append((template_score, feature_score))
#         print(f"Similarity score with {roi_descriptions[i]}: Template Score: {template_score:.4f}, Feature Matches: {feature_score}")

#     # Decision Logic: Choose the best score
#     best_match_index = np.argmax([score[0] for score in scores])
#     best_template_score = scores[best_match_index][0]
#     best_feature_score = scores[best_match_index][1]

#     # Determine if the best score corresponds to a valid or invalid orientation
#     if best_match_index in [0, 1]:  # Assuming 0 and 1 are valid indices
#         print("Kicker is in a valid orientation.")
#         return True
#     else:
#         print("Kicker is in an invalid orientation.")
#         return False

# # Full pipeline function
# def kicker_pipeline(image_path, valid_roi_paths, threshold=65, white_pixel_threshold_percentage=5):
#     # Step 1: Detect Kicker
#     current_roi, current_edges = detect_kicker(image_path, white_pixel_threshold_percentage=white_pixel_threshold_percentage)
#     if current_roi is None:
#         print("No kicker detected.")
#         return False  # No kicker detected

#     # Step 2: Check Orientation if kicker detected
#     return check_kicker_orientation(current_roi, valid_roi_paths)

# # Example usage
# valid_roi_paths = [
#     'Reference_Imgs/valid clean roi.jpg',
#     'Reference_Imgs/valid noisy roi.jpg',
#     'Reference_Imgs/invalid clean roi.jpg',
#     'Reference_Imgs/invalid noisy roi.jpg'
# ]

# #result = kicker_pipeline('OneDrive_2024-10-01/Images for Testing/Class 1 - Kicker at front position/cam2024_10_01_10_20_54.jpg', valid_roi_paths)

In [176]:
# import cv2
# import numpy as np

# # Function to extract the central vertical region from an image
# def extract_vertical_central_region(image, width_ratio=0.3, height_ratio=0.8):
#     height, width = image.shape
#     central_height = int(height * height_ratio)
#     central_width = int(width * width_ratio)
    
#     start_y = (height - central_height) // 2
#     start_x = (width - central_width) // 2
    
#     return image[start_y:start_y + central_height, start_x:start_x + central_width]

# # Function for template matching
# def template_matching(current_central, reference_central):
#     result = cv2.matchTemplate(current_central, reference_central, cv2.TM_CCOEFF_NORMED)
#     return np.max(result)

# # Function for feature matching using ORB
# def feature_matching(current_central, reference_central):
#     orb = cv2.ORB_create()
#     keypoints1, descriptors1 = orb.detectAndCompute(current_central, None)
#     keypoints2, descriptors2 = orb.detectAndCompute(reference_central, None)

#     # Use BFMatcher for matching
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
#     matches = bf.match(descriptors1, descriptors2)
    
#     return len(matches)  # Number of matches

# # Combined score calculation with weighted importance
# def combined_score(template_score, feature_score, w_template=0.5, w_feature=0.5):
#     return (w_feature * feature_score) + (w_template * template_score)

# # Function to check kicker orientation based on multiple reference images
# def check_kicker_orientation(current_roi, valid_roi_paths, width_ratio=0.3, height_ratio=0.8):
#     # Load all reference ROIs
#     valid_rois = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in valid_roi_paths]
    
#     if any(roi is None for roi in valid_rois):
#         print("Error: One or more reference ROIs not found.")
#         return False

#     # Extract central vertical regions
#     current_central = extract_vertical_central_region(current_roi, width_ratio, height_ratio)
#     central_regions = [extract_vertical_central_region(roi, width_ratio, height_ratio) for roi in valid_rois]

#     roi_descriptions = [
#         "clean valid ROI",
#         "noisy valid ROI",
#         "clean invalid ROI",
#         "noisy invalid ROI"
#     ]

#     # Calculate scores for each reference ROI
#     scores = []
#     for i, reference_central in enumerate(central_regions):
#         template_score = template_matching(current_central, reference_central)
#         feature_score = feature_matching(current_central, reference_central)
#         combined = combined_score(template_score, feature_score)
#         scores.append(combined)
#         print(f"ROI {roi_descriptions[i]} - Combined Score: {combined:.4f}, Template Score: {template_score:.4f}, Feature Matches: {feature_score}")

#     # Find the best match (highest combined score)
#     best_match_index = np.argmax(scores)
#     best_score = scores[best_match_index]

#     # Decision based on threshold
#     threshold = 20  # You can adjust this threshold based on testing
#     if best_score > threshold:
#         if best_match_index in [0, 1]:  # Valid ROIs
#             print(f"Kicker is in valid orientation (matched with {roi_descriptions[best_match_index]}).")
#             return True
#         else:
#             print(f"Kicker is in invalid orientation (matched with {roi_descriptions[best_match_index]}).")
#             return False
#     else:
#         print("No clear valid match, kicker might be in invalid orientation.")
#         return False

# # Full pipeline function
# def kicker_pipeline(image_path, valid_roi_paths, white_pixel_threshold_percentage=5):
#     # Step 1: Detect Kicker
#     current_roi, current_edges = detect_kicker(image_path, white_pixel_threshold_percentage=white_pixel_threshold_percentage)
#     if current_roi is None:
#         print("No kicker detected.")
#         return False  # No kicker detected

#     # Step 2: Check Orientation if kicker detected
#     return check_kicker_orientation(current_roi, valid_roi_paths)

# # Example usage
# valid_roi_paths = [
#     'Reference_Imgs/valid clean roi.jpg',
#     'Reference_Imgs/valid noisy roi.jpg',
#     'Reference_Imgs/invalid clean roi.jpg',
#     'Reference_Imgs/invalid noisy roi.jpg'
# ]

# #result = kicker_pipeline('OneDrive_2024-10-01/Images for Testing/Class 1 - Kicker at front position/cam2024_10_01_10_20_54.jpg', valid_roi_paths)


In [177]:
# import cv2
# import numpy as np

# # Function to extract the central vertical region from an image
# def extract_vertical_central_region(image, width_ratio=0.3, height_ratio=0.8):
#     height, width = image.shape
#     central_height = int(height * height_ratio)
#     central_width = int(width * width_ratio)
    
#     start_y = (height - central_height) // 2
#     start_x = (width - central_width) // 2
    
#     return image[start_y:start_y + central_height, start_x:start_x + central_width]

# # Function for template matching
# def template_matching(current_central, reference_central):
#     result = cv2.matchTemplate(current_central, reference_central, cv2.TM_CCOEFF_NORMED)
#     return np.max(result)

# # Function for feature matching using ORB
# def feature_matching(current_central, reference_central):
#     orb = cv2.ORB_create()
#     keypoints1, descriptors1 = orb.detectAndCompute(current_central, None)
#     keypoints2, descriptors2 = orb.detectAndCompute(reference_central, None)

#     if descriptors1 is None or descriptors2 is None:
#         return 0  # Return 0 if no descriptors found

#     # Use BFMatcher for matching
#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
#     matches = bf.match(descriptors1, descriptors2)
    
#     return len(matches)  # Number of matches

# # Combined score calculation with weighted importance
# def combined_score(template_score, feature_score, w_template=0.5, w_feature=0.5):
#     return (w_feature * feature_score) + (w_template * template_score)

# # Function to check kicker orientation based on multiple reference images
# def check_kicker_orientation(current_roi, valid_roi_paths, width_ratio=0.3, height_ratio=0.8):
#     # Load all reference ROIs
#     valid_rois = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in valid_roi_paths]
    
#     if any(roi is None for roi in valid_rois):
#         print("Error: One or more reference ROIs not found.")
#         return False

#     # Extract central vertical regions
#     current_central = extract_vertical_central_region(current_roi, width_ratio, height_ratio)
#     central_regions = [extract_vertical_central_region(roi, width_ratio, height_ratio) for roi in valid_rois]

#     roi_descriptions = [
#         "clean valid ROI",
#         "noisy valid ROI",
#         "clean invalid ROI",
#         "noisy invalid ROI"
#     ]

#     # Calculate scores for each reference ROI
#     valid_scores = []
#     invalid_scores = []
    
#     for i, reference_central in enumerate(central_regions):
#         template_score = template_matching(current_central, reference_central)
#         feature_score = feature_matching(current_central, reference_central)
#         combined = combined_score(template_score, feature_score)

#         if i in [0, 1]:  # Valid ROIs
#             valid_scores.append(combined)
#         else:  # Invalid ROIs
#             invalid_scores.append(combined)

#         print(f"ROI {roi_descriptions[i]} - Combined Score: {combined:.4f}, Template Score: {template_score:.4f}, Feature Matches: {feature_score}")

#     # Aggregate valid and invalid scores
#     total_valid_score = sum(valid_scores)
#     total_invalid_score = sum(invalid_scores)

#     print(f"Total Valid Score: {total_valid_score:.4f}, Total Invalid Score: {total_invalid_score:.4f}")

#     # Decision Logic: Compare aggregated scores
#     if total_valid_score > total_invalid_score:
#         print("Kicker is in valid orientation.")
#         return True
#     else:
#         print("Kicker is in invalid orientation.")
#         return False

# # Full pipeline function
# def kicker_pipeline(image_path, valid_roi_paths, white_pixel_threshold_percentage=5):
#     # Step 1: Detect Kicker
#     current_roi, current_edges = detect_kicker(image_path, white_pixel_threshold_percentage=white_pixel_threshold_percentage)
#     if current_roi is None:
#         print("No kicker detected.")
#         return False  # No kicker detected

#     # Step 2: Check Orientation if kicker detected
#     return check_kicker_orientation(current_roi, valid_roi_paths)

# # Example usage
# valid_roi_paths = [
#     'Reference_Imgs/valid clean roi.jpg',
#     'Reference_Imgs/valid noisy roi.jpg',
#     'Reference_Imgs/invalid clean roi.jpg',
#     'Reference_Imgs/invalid noisy roi.jpg'
# ]

# #result = kicker_pipeline('OneDrive_2024-10-01/Images for Testing/Class 1 - Kicker at front position/cam2024_10_01_10_20_54.jpg', valid_roi_paths)


## updated approach

In [178]:
import cv2
import numpy as np

# Function to extract the central vertical region from an image
def extract_vertical_central_region(image, width_ratio=0.3, height_ratio=0.8):
    height, width = image.shape
    central_height = int(height * height_ratio)
    central_width = int(width * width_ratio)
    
    start_y = (height - central_height) // 2
    start_x = (width - central_width) // 2
    
    return image[start_y:start_y + central_height, start_x:start_x + central_width]

# Function for template matching
def template_matching(current_central, reference_central):
    result = cv2.matchTemplate(current_central, reference_central, cv2.TM_CCOEFF_NORMED)
    return np.max(result)

# Function for feature matching using ORB
def feature_matching(current_central, reference_central):
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(current_central, None)
    keypoints2, descriptors2 = orb.detectAndCompute(reference_central, None)

    # Use BFMatcher for matching
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    
    return len(matches)  # Number of matches

# Function for histogram comparison
def histogram_comparison(current_central, reference_central):
    # Calculate histogram for both images
    hist_current = cv2.calcHist([current_central], [0], None, [256], [0, 256])
    hist_reference = cv2.calcHist([reference_central], [0], None, [256], [0, 256])

    # Normalize histograms
    hist_current = cv2.normalize(hist_current, hist_current)
    hist_reference = cv2.normalize(hist_reference, hist_reference)

    # Compare histograms using Chi-Square distance
    hist_score = cv2.compareHist(hist_current, hist_reference, cv2.HISTCMP_CHISQR)
    
    # Invert the score so that higher values represent better matches (since lower Chi-Square is better)
    return 1 / (1 + hist_score)  # This gives a score between 0 and 1

# Combined score calculation with weighted importance
def combined_score(template_score, feature_score, hist_score, w_template=0.4, w_feature=0.4, w_hist=0.2):
    return (w_template * template_score) + (w_feature * feature_score) + (w_hist * hist_score)

# Function to check kicker orientation based on multiple reference images
def check_kicker_orientation(current_roi, valid_roi_paths, width_ratio=0.3, height_ratio=0.8):
    # Load all reference ROIs
    valid_rois = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in valid_roi_paths]
    
    if any(roi is None for roi in valid_rois):
        print("Error: One or more reference ROIs not found.")
        return False

    # Extract central vertical regions
    current_central = extract_vertical_central_region(current_roi, width_ratio, height_ratio)
    central_regions = [extract_vertical_central_region(roi, width_ratio, height_ratio) for roi in valid_rois]

    roi_descriptions = [
        "clean valid ROI",
        "noisy valid ROI",
        "clean invalid ROI",
        "noisy invalid ROI"
    ]

    # Calculate scores for each reference ROI
    scores = []
    for i, reference_central in enumerate(central_regions):
        template_score = template_matching(current_central, reference_central)
        feature_score = feature_matching(current_central, reference_central)
        hist_score = histogram_comparison(current_central, reference_central)
        
        combined = combined_score(template_score, feature_score, hist_score)
        scores.append(combined)
        print(f"ROI {roi_descriptions[i]} - Combined Score: {combined:.4f}, Template Score: {template_score:.4f}, Feature Matches: {feature_score}, Histogram Score: {hist_score:.4f}")

    # Find the best match (highest combined score)
    best_match_index = np.argmax(scores)
    best_score = scores[best_match_index]

    # Decision based on threshold
    threshold = 20  # You can adjust this threshold based on testing
    if best_score > threshold:
        if best_match_index in [0, 1]:  # Valid ROIs
            print(f"Kicker is in valid orientation (matched with {roi_descriptions[best_match_index]}).")
            return True
        else:
            print(f"Kicker is in invalid orientation (matched with {roi_descriptions[best_match_index]}).")
            return False
    else:
        print("No clear valid match, kicker might be in invalid orientation.")
        return False

# Full pipeline function
def kicker_pipeline(image_path, valid_roi_paths, white_pixel_threshold_percentage=5):
    # Step 1: Detect Kicker
    current_roi, current_edges = detect_kicker(image_path, white_pixel_threshold_percentage=white_pixel_threshold_percentage)
    if current_roi is None:
        print("No kicker detected.")
        return False  # No kicker detected

    # Step 2: Check Orientation if kicker detected
    return check_kicker_orientation(current_roi, valid_roi_paths)

# Example usage
valid_roi_paths = [
    'Reference_Imgs/valid clean roi.jpg',
    'Reference_Imgs/valid noisy roi.jpg',
    'Reference_Imgs/invalid clean roi.jpg',
    'Reference_Imgs/invalid noisy roi.jpg'
]

# result = kicker_pipeline('OneDrive_2024-10-01/Images for Testing/Class 1 - Kicker at front position/cam2024_10_01_10_20_54.jpg', valid_roi_paths)


# Inference  (Ground Truth = Kicker Detected and in Valid Position)

In [179]:
import os
import shutil

def process_folder(folder_path, reference_roi_paths, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all the images in the folder
    for image_filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_filename)

        # Check if the file is an image (based on its extension)
        if image_filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            # Run the pipeline on the image to check orientation
            valid_orientation = kicker_pipeline(image_path, reference_roi_paths)

            # If kicker is in an invalid orientation, move the image to the output folder
            if not valid_orientation:
                print(f"Moving {image_filename} to '{output_folder}' (invalid kicker detected).")
                shutil.copy(image_path, os.path.join(output_folder, image_filename))
            else:
                print(f"{image_filename} is valid (kicker in correct orientation).")

        print("----------------------------------------------------------------------\n\n")

# Example usage
folder_path = 'Classes/class_0'
reference_roi_paths = [
    'Reference_Imgs/valid clean roi.jpg',
    'Reference_Imgs/valid noisy roi.jpg',
    'Reference_Imgs/invalid clean roi.jpg',
    'Reference_Imgs/invalid noisy roi.jpg'
]
output_folder = 'False_images_class0'

print("----------------------------------------------------------------------\n\n")
process_folder(folder_path, reference_roi_paths, output_folder)

----------------------------------------------------------------------


Test 1 failed. Checking white pixel coverage...
White Pixel Percentage in Upper Region: 16.43%
Kicker Detected by White Pixels
Kicker detected (based on white pixel coverage).
ROI clean valid ROI - Combined Score: 28.6051, Template Score: 0.0127, Feature Matches: 71, Histogram Score: 1.0000
ROI noisy valid ROI - Combined Score: 25.3985, Template Score: -0.0035, Feature Matches: 63, Histogram Score: 0.9995


ROI clean invalid ROI - Combined Score: 20.6075, Template Score: 0.0201, Feature Matches: 51, Histogram Score: 0.9971
ROI noisy invalid ROI - Combined Score: 25.4070, Template Score: 0.0175, Feature Matches: 63, Histogram Score: 0.9998
Kicker is in valid orientation (matched with clean valid ROI).
cam2024_09_26_17_44_30.jpg is valid (kicker in correct orientation).
----------------------------------------------------------------------


Test 1 failed. Checking white pixel coverage...
White Pixel Percentage in Upper Region: 31.37%
Kicker Detected by White Pixels
Kicker detected (based on white pixel coverage).
ROI clean valid ROI - Combined Score: 42.5981, Template Score: 0.0016, Feature Matches: 106, Histogram Score: 0.9875
ROI noisy valid ROI - Combined Score: 41.0015, Template Score: 0.0085, Feature Matches: 102, Histogram Score: 0.9906
ROI clean invalid ROI - Combined Score: 26.9981, Template Score: 0.0064, Feature Matches: 67, Histogram Score: 0.9779
ROI noisy invalid ROI - Combine

In [180]:
# import os
# import shutil

# def process_folder(folder_path, reference_roi_valid_path, reference_roi_invalid_path, output_folder):
#     # Create the output folder if it doesn't exist
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     # Iterate through all the images in the folder
#     for image_filename in os.listdir(folder_path):
#         image_path = os.path.join(folder_path, image_filename)

#         # Check if the file is an image (based on its extension)
#         if image_filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
#             #print(f"Processing {image_filename}...")

#             # Run the pipeline on the image to check orientation
#             valid_orientation = kicker_pipeline(image_path, reference_roi_valid_path, reference_roi_invalid_path)

#             # If kicker is in an invalid orientation, move the image to the output folder
#             if not valid_orientation:
#                 print(f"Moving {image_filename} to '{output_folder}' (invalid kicker detected).")
#                 shutil.copy(image_path, os.path.join(output_folder, image_filename))
#             else:
#                 print(f"{image_filename} is valid (kicker in correct orientation).")

# # Example usage
# folder_path = 'OneDrive_2024-10-01/Images for Testing/Class 1 - Kicker at front position'
# reference_roi_valid_path = 'Reference_Imgs/reference_roi_valid.jpg'
# reference_roi_invalid_path = 'Reference_Imgs/reference_roi_invalid.jpg'
# output_folder = 'False_images'

# process_folder(folder_path, reference_roi_valid_path, reference_roi_invalid_path, output_folder)
